# Creates Entity Embeddings

We will use the same requirements as in the end2end_ned_tutorial.ipynb file.

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
from pathlib import Path
import sys
import numpy as np
import logging
from importlib import reload
reload(logging)

reload(logging)
# Set to logging.DEBUG for more logging output
logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# root_dir = FILL IN FULL PATH TO DIRECTORY WHERE DATA IS DOWNLOADED (i.e., root_dir/data and root_dir/models)
root_dir = Path(".")
# entity_dir = FILL IN PATH TO ENTITY_DB DATA (i.e., tutorial_data/data
data_dir = root_dir / "data"
entity_dir = data_dir / "entity_db"
# model_dir = FILL IN PATH TO MODELS
model_dir = root_dir / "models"
device = 0

[2021-09-06 22:33:39,371][INFO] emmental.meta:122 - Setting logging directory to: /dfs/scratch0/lorr1/projects/bootleg/tutorial_data/data/log_dir
[2021-09-06 22:33:39,428][INFO] emmental.meta:64 - Loading Emmental default config from /dfs/scratch0/lorr1/projects/emmental/src/emmental/emmental-default-config.yaml.
[2021-09-06 22:33:39,429][INFO] emmental.meta:174 - Updating Emmental config from user provided config.
[2021-09-06 22:33:39,431][INFO] emmental.utils.seed:27 - Set random seed to 1234.
[2021-09-06 22:33:39,436][DEBUG] bootleg.end2end.bootleg_annotator:225 - Reading entity database
[2021-09-06 22:35:38,197][DEBUG] bootleg.end2end.bootleg_annotator:238 - Reading entity database
[2021-09-06 22:38:10,187][DEBUG] bootleg.end2end.bootleg_annotator:248 - Reading word tokenizers
[2021-09-06 22:38:10,196][DEBUG] urllib3.connectionpool:971 - Starting new HTTPS connection (1): huggingface.co:443
[2021-09-06 22:38:10,491][DEBUG] urllib3.connectionpool:452 - https://huggingface.co:443 "HE

Using Standard Cands CrossEntropy Loss


[2021-09-06 22:38:12,647][DEBUG] urllib3.connectionpool:452 - https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
[2021-09-06 22:38:12,657][DEBUG] urllib3.connectionpool:971 - Starting new HTTPS connection (1): huggingface.co:443
[2021-09-06 22:38:12,954][DEBUG] urllib3.connectionpool:452 - https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceCla

In [4]:
from bootleg.utils.parser.parser_utils import parse_boot_and_emm_args
from bootleg.utils.utils import load_yaml_file
from bootleg.extract_all_entities import run_model

config_in_path = model_dir / 'bootleg_uncased/bootleg_config.yaml'

config_args = load_yaml_file(config_in_path)

# set the model checkpoint path
config_args["emmental"]["model_path"] = str(model_dir / 'bootleg_uncased/bootleg_wiki.pth')

# set the path for the entity db and candidate map
config_args["data_config"]["entity_dir"] = str(entity_dir)

# set the devie if on CPU
config_args["emmental"]["device"] = device

# change dataloader threads to avoid memory issues with eval dataloader
config_args["data_config"]["dataloader_threads"] = 0

# save the new args (helps if you want to run things via command line)
config_args = parse_boot_and_emm_args(config_args)

print(f"Can also run\npython -m bootleg.extract_all_entities --config_script {model_dir}/bootleg_uncased/bootleg_config.yaml \
    --emmental.model_path {model_dir}/bootleg_uncased/bootleg_wiki.pth --data_config.entity_dir {entity_dir} \
    --emmental.device {device} --data_config.dataloader_threads 0")

In [5]:
bootleg_embedding_file = run_model(config=config_args)

In [ ]:
embs = np.load(bootleg_embedding_file)
print(embs.shape)

Let's see how to extact embeddings for specific entities

In [ ]:
from bootleg.symbols.entity_profile import EntityProfile

ep = EntityProfile.load_from_cache(entity_dir)

In [ ]:
qid = "Q21"
print("Title", ep.get_title(qid))

embedding = embs[ep.get_eid(qid)]

import faiss                   # make faiss available
index = faiss.IndexFlatL2(embs)

D, I = index.search(embedding, 10)